# TP 2 : RL

**Il est interdit d'utiliser la fonction 'compile' de TensorFlow pour ce TP**

Source utile : https://arxiv.org/pdf/1312.5602.pdf

Le but de ce TP sera de découvrir et mettre en place un algorithme d'apprentissage par renforcement profond : le DQN.
Vous apprendrez à un agent à jouer au jeu du cartpole, qui consiste à déplacer une plateforme sur laquelle est posé un baton de manière à ce que le baton reste en équilibre sur la plateforme.

Nous allons utiliser la librairie TensorFlow et faire tourner notre code sous Google Colab pour nos TP, ce qui aura l'avantage de nous permettre d'utiliser les GPUs mis à disposition gratuitement par Google.
Vous êtes libre d'utiliser une autre librairie que TensorFlow si vous en maîtrisez une autre ou votre machine personnelle si celle-ci possède un GPU suffisant, mais il est probable que les phases d'entraînement soit plus rapides sous Colab.

N'hésitez pas à vous référer aux docs de TensorFlow 2 et Keras disponibles sur internet lors de ce TP.

Ce TP s'effectue individuellement ou en binome. Veuillez respecter les consignes suivantes pour le rendu de votre travail :

*   Renommez le selon le format suivant : "DL_2023_TP_RL_prenom1_nom1_prenom2_nom2.ipynb".
*   Veillez à ce que votre nom et prénom soient complétés dans la cellule ci-dessous.
*   Veillez à avoir bien exécuté toutes les cellules de code et que les résultats soient tous bien visible dans le notebook sans nécessiter une ré-exécution.
*   Partagez le notebook avec ranvier.thomas.pro@gmail.com.

Si vous avez effectué le TP autrement que sur Google Colab :

*   Renommez le selon le format suivant : "DL_2023_TP_RL_prenom1_nom1_prenom2_nom2.ipynb".
*   Téléchargez le fichier ipynb.
*   Envoyez le fichier en pièce-jointe à ranvier.thomas.pro@gmail.com, en indiquant en tant qu'objet : "TP RL prenom1 nom1 prenom2 nom2".

---

## Veuillez compléter vos noms et prénoms ci-dessous :

*   **Prenom 1** : Yann
*   **Nom 1** : VINCENT
*   **Prenom 2** : Kévin
*   **Nom 2** : TANG

In [41]:
%%capture
import tensorflow as tf
import gym
from gym.wrappers.record_video import RecordVideo
import numpy as np
import random
import matplotlib.pyplot as plt
import time
from IPython.display import HTML
from base64 import b64encode
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

---
# Visualisation d'un épisode de Cartpole

Vous allez utiliser l'environement cartpole proposé par la librairie gym. A chaque étape l'environement nous envoie une observation correspondant à l'état actuel de l'environement et on doit envoyer une action (0 ou 1 pour gauche ou droite) en réponse.

Commencez par visualiser un épisode en ne sélectionnant que des actions aléatoires.

In [42]:
test = gym.make('CartPole-v1')

In [43]:
reset = test.reset()

In [44]:
reset

array([ 0.03062129, -0.02366157, -0.03908166, -0.00730987], dtype=float32)

In [45]:
# Environment initialization
env = gym.make('CartPole-v1')
# Put the environment in a wrapper to save the episode video
env = RecordVideo(env, './example')

# Start a new episode and get the first environment state
state = env.reset()
done = False
# While the episode is not over
while not done:
    # Select a random action
    action_index = random.randrange(env.action_space.n)
    # Execute selected action in the environment and get new observation and reward
    next_state, reward, done, _ = env.step(action_index)

env.close()

In [46]:
def viz_episode(filename):
    mp4 = open('./example/' + filename,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

    return HTML("""
    <video width=400 controls>
        <source src="%s" type="video/mp4">
    </video>
    """ % data_url)

# Play the episode video
viz_episode('rl-video-episode-0.mp4')

---
# Déclaration des hyperparamètres

Le but de ce TP n'étant pas de chercher les meilleurs hyperparamètres possibles vous trouverez ci-dessous l'initilisation d'hyperparamètres par défaut qui donnent de bons résultats.

In [47]:
# Reinitialize environment
env = gym.make('CartPole-v1')

# Get environment observation size and number of possible actions
obs_size = env.observation_space.shape[0]
nb_actions = env.action_space.n

#Hyper Parameters
training_episodes = 200                     # Max number of training episodes
gamma = .95                                 # Discount factor used in Q-function
learning_rate = .001                        # Learning rate of the optimizer
batch_size = 24
epsilon_max = 1.                            # Maximum epsilon value at 1.
epsilon_min = .05                           # Minimum epsilon value at .05
epsilon_decay = .995                        # Epsilon decay factor at .995
replay_memory_capacity = 2500               # Limit memory capacity at 2500
print_step = 10                             # Print recap every 10 steps
target_avg_reward = 400                     # Early stop if avg reward >= 200

---
# Création de la replay memory

La replay memory est utilisée pour stocker les transitions entre l'agent et l'environement. Chaque transition stockée est composée de l'état de départ, l'action réalisée par l'agent, l'état d'arrivée, le reward obtenu et si l'état d'arrivée est terminal ou non.

In [48]:
class ReplayMemory:
    def __init__(self, capacity):
        self._capacity = capacity
        self._memory = []

    def push(self, state, next_state, action, reward, done):
        """
        Cette méthode doit permettre de stocker une transition dans la mémoire
        tout en s'assurant que le nombre d'transitions stockée ne dépasse pas
        sa capacité.
        Si c'est le cas la nouvelle transition doit écraser la plus ancienne.

        Une transition contient :
            - state: np.array, l'état de départ
            - next_state: np.array, l'état d'arrivée
            - action: tf.tensor, l'action effectuée. Dans un soucis d'efficience
                      vous devez stocker l'action directement sous forme de
                      one-hot tensor, obtenu avec : action = tf.one_hot(action_index, nb_actions)
            - reward: int, le reward obtenu grâce à l'action effectuée
            - done: boolean, indique si le baton est tombé ou non
        """
        transition = (state, next_state, action, reward, done)

        if len(self._memory) > self._capacity:
          self._memory.pop(0)
        self._memory.append(transition)


    def sample(self, batch_size):
        """
        Cette méthode doit extraire une minibatch de la taille voulue de la
        mémoire de manière aléatoire. Elle prend en entrée la taille de la batch
        à extraire.

        La méthode doit retourner :
            - state_s: np.array, contenant les états de départ
            - state_next_s: np.array, contenant les états d'arrivées
            - action_s: tf.tensor, contenant les one-hots vectors des actions
            - rewards_s: tf.tensor, contenant les rewards
            - opp_done_s: tf.tensor, contenant l'opposé du flag 'done' converti en float
        """
        samples = random.sample(self._memory, batch_size)
        state_s, state_next_s, action_s, rewards_s, opp_done_s = zip(*samples)

        state_s = np.array(state_s)
        state_next_s = np.array(state_next_s)
        action_s = tf.convert_to_tensor(action_s, dtype=tf.float32)
        rewards_s = tf.convert_to_tensor(rewards_s, dtype=tf.float32)
        opp_done_s = tf.cast(1 - np.array(opp_done_s), dtype=tf.float32)

        return state_s, state_next_s, action_s, rewards_s, opp_done_s

    def __len__(self):
        """
        Cette méthode override les appels de length sur l'objet ReplayMemory.
        Elle doit retourner la quantité actuelle de transitions sauvegardées
        dans la mémoire.
        """
        return len(self._memory)

---
# Création de l'agent

L'agent apprend à intéragir et executer ses actions avec l'environnement.

In [107]:
class Agent():
    def __init__(self, replay_memory, model, optimizer, loss_function, env):
        self._rm = replay_memory
        self._epsilon = epsilon_max
        self._model = model
        self._optimizer = optimizer
        self._loss_function = loss_function
        self._env = env
        # History
        self._reward_hist = []

    def train(self):
        """
        Méthode principale, lance l'entraînement de l'agent.
        """
        start = time.time()
        avg_reward = 0
        for episode in range(training_episodes):
            # Run 1 episode
            episode_reward = self._run_episode(episode)
            self._reward_hist.append(episode_reward)
            avg_reward += episode_reward
            if (episode + 1) % print_step == 0:
                print(f'episode: {episode + 1}/{training_episodes}, avg reward: {avg_reward / print_step}, e: {self._epsilon}')
                # Early stop
                if avg_reward / print_step >= target_avg_reward:
                    break
                avg_reward = 0
        print(f'training done in {time.time() - start:.2f}s')
        self._plot_history()

    def _run_episode(self, num_episode):
        """
        Cette méthode doit executer un épisode d'apprentissage.
        """
        # Initialize environment and get first state
        # convert the state to np.array type
        state = np.array(self._env.reset())

        episode_reward = 0
        done = False
        while not done:
            # Select an action, randomly in epsilon % of the time, and using
            # the model output otherwise.
            if np.random.rand() < self._epsilon :
                action_index = np.random.randint(self._env.action_space.n)
            else:
                model_output = self._model(np.expand_dims(state, axis=0))
                action_index = tf.argmax(model_output[0]).numpy()
            # Obtain the model output: self._model(np.expand_dims(state, axis=0))
            # You need to convert the model output which is a score for each
            # possible action to the index of the action for which the model
            # predicts the highest score.


            # Perform selected action in the environment, get next state, reward
            # and done status
            next_state, reward, done, _ = self._env.step(action_index)

            # Convert the next state to np.array type
            next_state = np.array(next_state)

            # Update episode total reward
            angle = next_state[2]
            episode_reward += reward*(0.1 * abs(angle))
            if abs(angle)>5:
              episode_reward -= 50

            # Convert the selected action index to a one-hot tensor for
            # efficiency (cf. tf.one_hot())
            action_one_hot = tf.one_hot(action_index, self._env.action_space.n)

            # Add observation to the replay memory (cf. self._rm.push)
            self._rm.push(state, next_state, action_one_hot, reward, done)

            # Replace current state with next state
            state = next_state

            # If there are enough transitions in the replay memory:
            #    - extract a minibatch from the memory
            #    - call the optimize function to train the model on the minibatch
            #    - update epsilon
            if len(self._rm) >= batch_size:
                # Extract a minibatch from the memory
                batch = self._rm.sample(batch_size)

                # Unpack the minibatch
                state_s, state_next_s, action_s, rewards_s, opp_done_s = batch

                # Call the optimize function to train the model on the minibatch
                self._optimize(state_s, state_next_s, action_s, rewards_s, opp_done_s)

                # Update epsilon (you need to define how you want to update it)
                self._epsilon = max(epsilon_min, self._epsilon * epsilon_decay)

        return episode_reward

    @tf.function
    def _optimize(self, state_s, state_next_s, action_s, rewards_s, opp_done_s):
        """
        Cette méthode calcule la Q-loss, elle doit ensuite faire la
        backpropagation.
        """
        # Q-loss
        q_values_target = self._model(state_next_s)
        q_values_target = rewards_s + (gamma * tf.reduce_max(q_values_target, axis=1)) * opp_done_s
        with tf.GradientTape() as tape:
            q_values_pred = self._model(state_s)
            q_values_pred = tf.reduce_sum(tf.multiply(q_values_pred, action_s), axis=1)
            loss = self._loss_function(q_values_target, q_values_pred)

        # Backpropagation (compute gradients and apply gradients using tf functions)
        gradients = tape.gradient(loss, self._model.trainable_variables)
        self._optimizer.apply_gradients(zip(gradients, self._model.trainable_variables))

    def _plot_history(self):
        plt.figure(figsize=(10, 5))
        plt.plot(self._reward_hist)
        plt.xlabel('episodes')
        plt.ylabel('reward')
        plt.title('Reward per episode')
        plt.show()

    def real_episode(self, env):
        state = env.reset()
        done = False
        while not done:
            output = self._model(np.expand_dims(state, axis=0))
            action_index = np.argmax(output.numpy())
            state, _, done, _ = env.step(action_index)

---
# Entraînement de l'agent

Si tout est correct vous devriez voir la récompense moyenne des 10 derniers épisodes d'apprentissage commencer à augmenter entre 50 et 100 épisodes.

In [108]:
# Define a fully-connected model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(obs_size,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(nb_actions, activation='linear')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss_function = tf.keras.losses.MeanSquaredError()
replay_memory_capacity = 10000
replay_memory = ReplayMemory(replay_memory_capacity)

# Create the agent
agent = Agent(replay_memory, model, optimizer, loss_function, env)

In [ ]:
# Training
agent.train()

episode: 10/200, avg reward: 0.12666030576801862, e: 0.49571413690105054
episode: 20/200, avg reward: 0.10254640484694391, e: 0.26359640222486147
episode: 30/200, avg reward: 0.2862352574180113, e: 0.05
episode: 40/200, avg reward: 0.6707070448316518, e: 0.05
episode: 50/200, avg reward: 1.1694971013763682, e: 0.05
episode: 60/200, avg reward: 0.72751878515588, e: 0.05
episode: 70/200, avg reward: 0.6828846514449313, e: 0.05


In [103]:
# Run a test episode
env = gym.make('CartPole-v1')
env = RecordVideo(env, './episode')
agent.real_episode(env)
env.close()

In [104]:
# Visualize the test episode
viz_episode('../episode/rl-video-episode-0.mp4')

In [94]:
# Visualize the test episode
viz_episode('../episode/rl-video-episode-1.mp4')

In [95]:
# Visualize the test episode
viz_episode('../episode/rl-video-episode-125.mp4')